<a href="https://colab.research.google.com/github/tleitch/BDML/blob/main/scholastic_travel/stochastic_travel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fancyimpute

In [2]:
import pandas as pd
import xlrd
pd.options.mode.chained_assignment = None
#from fancyimpute import MICE
from fancyimpute import IterativeImputer as MICE
import copy
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
## sometimes in data ,people represent NA values differently. Pandas provide a way to input all possible NA values in form of list so that they are stored in memory as NA##
exhibit_1 = pd.read_csv("https://raw.githubusercontent.com/tleitch/BDML/main/scholastic_travel/exhibit_1.csv", na_values=["#N/A", "#N/A N/A", "#NA", "-1.#IND", "-1.#QNAN", "-NaN", "-nan", "1.#IND", "1.#QNAN", "N/A", "NA", "NULL", "NaN", "nan"])
exhibit_2 = pd.read_csv("https://raw.githubusercontent.com/tleitch/BDML/main/scholastic_travel/exhibit_2.csv",na_values = ["#N/A", "#N/A N/A", "#NA", "-1.#IND", "-1.#QNAN", "-NaN", "-nan", "1.#IND", "1.#QNAN", "N/A", "NA", "NULL", "NaN", "nan"])

exhibit_1 = exhibit_1[1:2389]
exhibit_2 = exhibit_2[1:2389]

In [ ]:
exhibit_1.head()

In [ ]:
exhibit_1.dtypes

In [ ]:
exhibit_2.head()

In [ ]:
exhibit_2.dtypes

In [ ]:
exhibit_1.shape

In [ ]:
df_merged = pd.merge(exhibit_1, exhibit_2, on="ID")

In [ ]:
df_merged["cancellation"] = df_merged["FRP.Cancelled"]/df_merged["FRP.Active"]

In [ ]:
date_cols = ["Departure.Date","Return.Date","Deposit.Date","Early.RPL","Latest.RPL","Initial.System.Date","FirstMeeting","LastMeeting"]



In [ ]:
for column in date_cols:
    df_merged[column]=pd.to_datetime(df_merged[column])

In [ ]:
df_merged["Return.Date"]

In [ ]:

unchanged_cols = ["Is.Non.Annual.","Days","Tuition","School.Type","Parent.Meeting.Flag","MDR.High.Grade","Income.Level","School.Sponsor","SPR.New.Existing","NumberOfMeetingswithParents","DifferenceTraveltoLastMeeting","SchoolGradeTypeHigh","FPP.to.School.enrollment","FPP.to.PAX","SchoolSizeIndicator","Retained.in.2012."]
df_final1 = df_merged[unchanged_cols]

In [ ]:
df_final1["frp.active"]  =df_merged["FRP.Active"]/df_merged["FPP"]

df_final1["cancellation"] = df_merged["FRP.Cancelled"]/df_merged["FRP.Active"]
df_final1["cancelled.PAX"] = df_merged["Cancelled.Pax"]/df_merged["FPP"]

In [ ]:
MAX_VALUE = 100000000

In [ ]:
df_final1['fpp'] = pd.cut(x=df_merged['FPP'], bins=[-MAX_VALUE,10,20,35,MAX_VALUE],labels=['small','critical','safe','large'])
df_final1["trend"]=((df_merged.iloc[:, 56:59].mean(axis=1,skipna=True) > df_merged.iloc[:,56:58].mean(axis=1,skipna=True)).astype(int)).astype("category")

In [ ]:
object_columns = df_final1.select_dtypes(include='object').columns
category_columns=df_final1.select_dtypes(include='category').columns
float_columns=df_final1.select_dtypes(include='float64').columns

In [ ]:
df_final1.dtypes

In [ ]:
for column in object_columns:
    df_final1[column] = df_final1[column].astype("category")


## Converting the data to one-hot encoding so that modelling can be done. ##

In [ ]:
df_final1 = pd.get_dummies(df_final1)

In [ ]:
df_final1.dtypes

In [ ]:
df_final1[float_columns].describe()

In [ ]:
# In the above dataframe, we saw cancellation has infinity values which is not needed, so we drop the colum
df_final1.drop(columns="cancellation",inplace=True)

In [ ]:
## Chechking missing values in every column ##
print (df_final1.isnull().sum())

## Imputation of missing values ##

In [ ]:
imputer=MICE().fit_transform(df_final1)
df_final_imputed = pd.DataFrame(imputer, columns=df_final1.columns)

## Checking missing value again, now we see missing values as zero ##

In [ ]:
print (df_final_imputed.isnull().sum())

In [ ]:
df_final_imputed_copy=copy.deepcopy(df_final_imputed)

## Scaling the data ##

In [ ]:
scaler = MinMaxScaler()
float_columns=df_final_imputed_copy.select_dtypes(include='float64').columns
df_final_imputed_copy[float_columns] = scaler.fit_transform(df_final_imputed_copy[float_columns])

## Forward regression ##

In [ ]:
def forward_regression(X, y,
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    initial_list = []
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add   with p-value '.format(best_feature, best_pval))

        if not changed:
            break

    return included

In [ ]:
df_final_imputed_copy["Retained"] = df_final_imputed_copy["Retained.in.2012."]
df_final_imputed_copy.drop(columns="Retained.in.2012.",inplace=True)

In [ ]:
X_train= df_final_imputed_copy.drop(columns="Retained")
Y_train= df_final_imputed_copy["Retained"]

In [ ]:
pd.value_counts(df_final_imputed_copy["Retained"])

In [ ]:
pd.value_counts(df_merged["SchoolGradeTypeHigh"])                                       

In [ ]:
forward_regression(X_train,Y_train)

## Generalized linear model ##

In [ ]:
import statsmodels.api as sm

exog = sm.add_constant(X_train)
endog=Y_train
gamma_model = sm.GLM(endog,exog,family=sm.families.Gamma())

gamma_results = gamma_model.fit()

print(gamma_results.summary())

In [ ]:
sm.add_constant(X_train)